In [ ]:
import getpass
from dalle.imperative.api.labs import get_bearer_token

print("Attempting to get token for DALL-E...")
token = get_bearer_token(
    input("OpenAI Username: "),
    getpass.getpass('OpenAI Password: ')
)
print("Token:", token[:5] + ("*" * (len(token) - 5)))

In [ ]:
from dalle.imperative.api.labs import get_tasks

from_ts = 0
print("Getting tasks for tasks starting from timestamp", from_ts)
tasks = get_tasks(token, from_ts)
print("# of Tasks:", len(tasks))

In [ ]:
from typing import List
import matplotlib.pyplot as plt
from PIL import Image as PILImage
from PIL.Image import Image as PILImageType
from urllib.request import urlopen
import textwrap, os

def display_images(
    images: List[PILImageType],
    columns=4, width=20, height=8, max_images=24,
    label_wrap_length=50, label_font_size=8,
    disable_axis=True):

    if not images:
        print("No images to display.")
        return

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]

    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):

        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        if disable_axis:
            plt.axis('off')
        plt.imshow(image)

        if hasattr(image, 'filename'):
            title=image.filename
            if title.endswith("/"): title = title[0:-1]
            title=os.path.basename(title)
            title=textwrap.wrap(title, label_wrap_length)
            title="\n".join(title)
            plt.title(title, fontsize=label_font_size)

def generation_to_pil(g):
    img = urlopen(g.generation.image_path)
    return PILImage.open(img)

def display_task_generations(t):
    images = []
    for generation in t.generations:
        images.append(generation_to_pil(generation))
    display_images(images)

In [ ]:
if tasks:
    display_task_generations(tasks[0])
else:
    print("No tasks found.")

In [ ]:
from dalle.imperative.api.labs import create_text2im_task

print("Attempting to create text2im task...")
pending_task = create_text2im_task(token, "A cute cat")
print(pending_task)

In [ ]:
from dalle.imperative.api.labs import poll_for_task_completion

print("Waiting for task to complete...")
task = poll_for_task_completion(token, pending_task.id)
display_task_generations(task)

In [ ]:
from dalle.imperative.api.labs import create_variations_task

print("Attempting to create variations task...")
pending_task = create_variations_task(token, task.generations[0].id)
print(pending_task)

In [ ]:
print("Waiting for task to complete...")
task = poll_for_task_completion(token, pending_task.id)
display_task_generations(task)

In [ ]:
import base64
from io import BytesIO

# Convert generation to a cropped base64 PNG string
image = generation_to_pil(task.generations[0])
image = image.convert("RGBA")
for i in range(image.width):
    if i > image.width / 2:
        for j in range(image.height):
            image.putpixel((i, j), (0, 0, 0, 0))
with BytesIO() as buffer:
    image.save(buffer, format="PNG")
    base64_png = base64.b64encode(buffer.getvalue()).decode()
image

In [ ]:
from dalle.imperative.api.labs import create_inpainting_task


print("Attempting to create inpainting task...")
pending_task = create_inpainting_task(token, caption="A cute cat, with a dark side",
                                          masked_image=base64_png,
                                          parent_id_or_image=task.generations[0].id)
print(pending_task)

In [ ]:
print("Waiting for task to complete...")
task = poll_for_task_completion(token, pending_task.id)
display_task_generations(task)